In [2]:
import glob, os
from astropy.io import ascii
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, vstack
import matplotlib as mpl
from astropy import units as u
import speclite.filters
import time
import sncosmo

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"

mpl.rcParams["axes.titlesize"] = 14
mpl.rcParams["axes.labelsize"] = 20
plt.rcParams['savefig.dpi'] = 200
plt.rc('font', family='serif')

In [30]:
from helper import makeSpecColors
from helper import convert_flam2fnu
from helper import get_wollaeger
from helper import convert_app2abs
from helper import convert_abs2app
kncbtbl = get_wollaeger()
from helper import get_bandwidth_table
bdwtbl = get_bandwidth_table()
#	speclite
from helper import get_speclite_med
meds = get_speclite_med()
mlam = meds.effective_wavelengths
mbdw = bdwtbl['bandwidth'][bdwtbl['group']=='Med']*u.Angstrom
from helper import get_speclite_sdss
sdss = get_speclite_sdss()
slam = sdss.effective_wavelengths
sbdw = bdwtbl['bandwidth'][bdwtbl['group']=='SDSS']*u.Angstrom
from helper import get_speclite_jc
jc = get_speclite_jc()
jclam = jc.effective_wavelengths
jcbdw = bdwtbl['bandwidth'][bdwtbl['group']=='Johnson Cousin']*u.Angstrom
from helper import get_speclite_lsst
lsst = get_speclite_lsst()
lsstlam = lsst.effective_wavelengths
from helper import get_lsst_depth
from helper import get_kmtnet_depth
from helper import get_7dt_depth
dptbl = get_7dt_depth()
from helper import get_7dt_broadband_depth

from helper import get_speclite_lsst
lsst = get_speclite_lsst()
lsstlam = lsst.effective_wavelengths
from helper import get_lsst_depth
from helper import get_kmtnet_depth
from helper import get_lsst_bandwidth
lsstbdw = get_lsst_bandwidth()

In [5]:
from scipy.optimize import curve_fit

def func(x, a):
	return a*x

def calc_chisquare(obs, exp):
	return np.sum((obs-exp)**2/exp)

def calc_redchisquare(obs, exp, dof):
	return calc_chisquare(obs, exp)/dof

def calc_AIC(k, L):
	"""
	L : likelihood
	k : the number of free parameters
	"""
	AIC = 2*k-2*np.log(L)
	return AIC

def calc_AICc(AIC, k, n):
	"""
	n : the number of filters
	k : the number of free parameters
	"""
	AICc = AIC+(2*k*(k+1))/(n-k-1)
	return AICc

def calc_chisq_det(fobs, fobserr, fmdl):
	"""
	Chisquare calculcation for detection
	fobs     : the observed flux in the nth band
	fobsderr : the uncentainty of fobs
	fmdl     : the model flux
	"""
	chisq = ((fobs-fmdl)/fobserr)**2
	return chisq

def calc_chisq_nd(fobs, fobserr, fmdl, flim):
	"""
	Chisquare calculcation for non-detection
	fobs     : the observed flux in the nth band
	fobsderr : the uncentainty of fobs
	fmdl     : the model flux
	flim     : the upper limit of flux in the nth band
	"""
	from scipy import special
	chisq = -2*np.log(np.sqrt(np.pi/2)*fobserr*(1+special.erf((flim-fmdl)/(np.sqrt(2)*fobserr))))
	return chisq


def calc_mchisquare(fobs, fobserr, fmdl, flim):
	"""
	Modified chisquare
	"""
	mchisq = np.sum(calc_chisq_det(fobs, fobserr, fmdl))+np.sum(calc_chisq_nd(fobs, fobserr, fmdl, flim))
	return mchisq

def calc_l(AICc0, AICc1):
	l = np.exp(-0.5*(AICc0-np.array([AICc0, AICc1]).min()))
	return l

def calc_AICc_weight(AICc0, AICc1):
	"""
	l0 : standard
	w>0.99 : case 0 is 100 times more likey to be the best model than case 1
	"""
	l0 = calc_l(AICc0, AICc1)
	l1 = calc_l(AICc1, AICc1)
	w = l0/(l0+l1)
	return w


In [41]:
def extract_fnu_from_simulation(psimtbl):
	fnuabs = np.array([(psimtbl[f"{filte}"].item()*u.ABmag).to(u.uJy).value for filte in psimtbl.keys() if 'm' in filte])
	return fnuabs

def extract_fnu_anwr_from_observation(pobstbl):
	fnuabs_anwr = np.array([(pobstbl[f"{filte}"].item()*u.ABmag).to(u.uJy).value for filte in pobstbl.keys() if 'magabs' in filte])
	return fnuabs_anwr

def extract_fnu_from_observation(pobstbl):
	fnuobs = np.array([pobstbl[f"{filte}"].item() for filte in pobstbl.keys() if 'fnuobs_' in filte])
	fnuobserr = np.array([pobstbl[f"{filte}"].item() for filte in pobstbl.keys() if 'fnuerr_' in filte])
	return fnuobs, fnuobserr


# Main

- Initical setting

In [36]:
d = 40 # [Mpc]
mfilterlist = [f"m{n}" for n in np.arange(400, 875+25, 25)]
mdepthlist = (dptbl['5sigma_depth'].value*u.ABmag).to(u.uJy)

In [8]:
obsphtlist = sorted(glob.glob(f"../5.result/kn_sim_cube_obs/med_iter100_{d}Mpc/Run*med.ecsv"))
cmpphtlist = sorted(glob.glob('../3.table/sn_sim_sncosmo_synphot/*_med.ecsv'))

print(f'KN obs. data    : {len(obsphtlist)}')
print(f'Other sim. data : {len(cmpphtlist)}')

KN obs. data    : 6300
Other sim. data : 50


In [11]:
ii = -8
jj = -1

obspht = obsphtlist[ii]
cmppht = cmpphtlist[jj]

print(f"[{ii}] Input obs   : {os.path.basename(obspht)}")
print(f"[{jj}] Comp. Model : {os.path.basename(cmppht)}")

#	Read tables
obstbl = ascii.read(obspht)
cmptbl = ascii.read(cmppht)

[-8] Input obs   : Run_TS_dyn_all_lanth_wind2_all_md0.1_vd0.3_mw0.1_vw0.15_angle90_synphot_med_obs_d40_iter100.med.ecsv
[-1] Comp. Model : v19-2009ip-corr_v1.0_type_IIn_sn_z1.0_synphot_med.ecsv


In [76]:
t = 1.0
seed = 1
t_cmp = 10

#	Part obs. Table
pobstbl = obstbl[
	(obstbl['t']==t) &
	(obstbl['seed']==seed)
]

pcmptbl = cmptbl[
	#	Supernova Type Ia
	(cmptbl['t']==t_cmp)
]

fnuabs = extract_fnu_from_simulation(pcmptbl)
fnuabs_anwr = extract_fnu_anwr_from_observation(pobstbl)
fnuobs, fnuobserr = extract_fnu_from_observation(pobstbl)

indx_dt = np.array([],  dtype=np.int8)
indx_nd = np.array([],  dtype=np.int8)

for ff, filte in enumerate(mfilterlist):
	snr = pobstbl[f'snr_{filte}'].item()
	if snr < 5:
		indx_nd = np.append(indx_nd, int(ff))
	else:
		indx_dt = np.append(indx_dt, int(ff))

print(f"Detection : {indx_dt}")
print(f"n/d       : {indx_nd}")


fobs = fnuobs[indx_dt]
fobserr = fnuobserr[indx_dt]
fmdl = fnuabs[indx_dt]

ainit = np.median(fobs)/np.median(fmdl)


# np.median(fnuobs[indx_dt])

# def calc_mchisquare(fobs, fobserr, fmdl, flim):
# 	"""
# 	Modified chisquare
# 	"""
# 	mchisq = np.sum(calc_chisq_det(fobs, fobserr, fmdl))+np.sum(calc_chisq_nd(fobs, fobserr, fmdl, flim))
# 	return mchisq

Detection : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
n/d       : [14 15 16 17 18 19]


279.96211434696835

In [70]:
aa = np.array([0, 1, 2])
np.where(aa==0)

(array([0]),)